In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
import numpy as np

plt.style.use('paper.mplstyle')

In [2]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'
table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue.fits' %root_dir)
table_assoc = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations.fits' %root_dir)
table_muse = QTable.read('%s/data_misc/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits' %root_dir)

table = table[table['HII_class_v3_MUSE']==1]
table_assoc = table_assoc[table_assoc['HII_class_v3_MUSE']==1]
table_muse = table_muse[table_muse['HII_class_v3']==1]

table_assoc = table_assoc['gal_name', 'region_ID', 'region_ID_association_nuv_32pc', 'age_association_nuv_32pc', 'mass_association_nuv_32pc']
table_assoc = table_assoc[np.isfinite(table_assoc['age_association_nuv_32pc'])]
table_assoc = table_assoc[table_assoc['mass_association_nuv_32pc']>1]

In [3]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]
columns = ['gal_name'] + [f'percentile_{p}' for p in percentiles]
data = []

for gal_name in np.unique(table['gal_name']):

    table_masked = table[table['gal_name'] == gal_name]
    table_muse_masked = table_muse[table_muse['gal_name'] == gal_name]

    gal_data = [gal_name.replace('NGC', 'NGC~').replace('IC', 'IC~')]

    p_age = np.nanpercentile(table_assoc[table_assoc['gal_name'] == gal_name]['age_association_nuv_32pc'], percentiles)
    p_mass = np.log10(np.nanpercentile(table_assoc[table_assoc['gal_name'] == gal_name]['mass_association_nuv_32pc'], percentiles))

    for values in [p_age, p_mass]:
        for value in values:
            gal_data.extend([value])
    
    data.append(gal_data)

/Users/abarnes/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  arr.partition(


In [4]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]

gal_data = ['All']

p_age = np.nanpercentile(table_assoc['age_association_nuv_32pc'], percentiles)
p_mass = np.log10(np.nanpercentile(table_assoc['mass_association_nuv_32pc'], percentiles))

for values in [p_age, p_mass]:
    for value in values:
        gal_data.extend([value])

data.append(gal_data)

In [5]:
# Create a new QTable with the calculated percentiles
names1 = ['gal_name']
names2 = [f'p{p}_age' for p in percentiles]
names3 = [f'p{p}_mass' for p in percentiles]

names = names1 + names2 + names3
new_table = QTable(rows=data, names=names)

new_table

gal_name,p10_age,p50_age,p90_age,p10_mass,p50_mass,p90_mass
str8,float64,float64,float64,float64,float64,float64
IC~5332,1.0,2.0,8.0,2.8379570675946844,3.4800973852244064,4.128659726422383
NGC~0628,1.0,4.0,12.400000000000013,2.9525689873082217,3.6296468393766115,4.549223763244159
NGC~1087,1.0,3.0,8.0,3.3068673711771446,4.029733282857735,4.892938775292047
NGC~1300,1.0,4.0,6.0,2.999644471702198,3.7683482870234344,4.460208507428614
NGC~1365,1.0,3.0,8.0,3.5532092172967005,4.3745463441416454,5.314378196740826
NGC~1385,1.0,5.0,12.800000000000011,3.4747450122452235,4.18038187498812,5.000209650617437
NGC~1433,1.0,1.0,6.0,3.1694970512502625,4.051695751371738,4.577443802923829
NGC~1512,1.0,1.0,7.000000000000001,3.085178481054135,3.869973414756753,4.48555698294015
NGC~1566,1.0,3.0,8.0,3.3443214371648944,4.083982388364554,4.953263372351347


In [6]:
names = ['gal_name',
        'p10_age',
        'p50_age',
        'p90_age',
        'p10_mass',
        'p50_mass',
        'p90_mass'
        ]

formats={
        'gal_name': '%s',
        'p10_age': '%.1f',
        'p50_age': '%.1f',
        'p90_age': '%.1f',
        'p10_mass': '%.2f',
        'p50_mass': '%.2f',
        'p90_mass': '%.2f'
        }

new_table.write('./tabs/agemass_props.tex', formats=formats, names=names, overwrite=True)
# new_table.write('./tabs/lumrad_props.fits', overwrite=True)
# new_table.write('./tabs/lumrad_props.dat', format='ascii', overwrite=True)